In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [11]:
train_data_dir = r'C:\Users\Asif_\Desktop\dogs-vs-cats\train'
test_data_dir = r'C:\Users\Asif_\Desktop\dogs-vs-cats\test1'

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Scale pixel values to the range [0, 1]
    rotation_range=40,      # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,        # Randomly apply shear transformations
    zoom_range=0.2,         # Randomly zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill in missing pixels with the nearest available pixel
)

In [13]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 25000 images belonging to 1 classes.


In [15]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 12500 images belonging to 1 classes.


In [32]:
from tensorflow.keras import regularizers

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

# Add a dropout layer with 0.5 dropout rate
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Define learning rate schedule
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define model checkpointing
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=30,
    validation_data=test_generator,
    validation_steps=100,
    callbacks=[reduce_lr, early_stopping, model_checkpoint]
)

Epoch 1/30
200/200 [==============================] - 121s 601ms/step - loss: 0.4815 - accuracy: 0.9998 - val_loss: 0.0021 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/30


C:\Users\Asif_\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


200/200 [==============================] - 119s 595ms/step - loss: 0.6684 - accuracy: 0.9965 - val_loss: 1.9988 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/30
200/200 [==============================] - 130s 651ms/step - loss: 1.7140 - accuracy: 1.0000 - val_loss: 1.5080 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/30
200/200 [==============================] - 124s 622ms/step - loss: 1.3679 - accuracy: 1.0000 - val_loss: 1.2427 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/30
200/200 [==============================] - 116s 580ms/step - loss: 1.1413 - accuracy: 1.0000 - val_loss: 1.0473 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/30
200/200 [==============================] - 115s 574ms/step - loss: 0.9677 - accuracy: 1.0000 - val_loss: 0.8929 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/30
200/200 [==============================] - 116s 578ms/step - loss: 0.8788 - accuracy: 1.0000 - val_loss: 0.8643 - val_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 8/30
200/200 [==============================] -

In [24]:
!pip install pillow

In [44]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=50)
print(f'Test accuracy: {test_accuracy}')

# Make predictions on individual images
# Assuming you have a test image in 'path_to_test_image'
from tensorflow.keras.preprocessing import image

img_path = r'C:\Users\Asif_\Desktop\Doggy_image\download (1).jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)

if prediction[0] > 0.5:
    print('Cat')
else:
    print('Dog')

50/50 [==============================] - 6s 128ms/step - loss: 0.0021 - accuracy: 1.0000
Test accuracy: 1.0
1/1 [==============================] - 0s 41ms/step
Dog
